<a href="https://colab.research.google.com/github/Hellfik/data-augmentation/blob/main/Copy_of_AugLy_audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Note: restart runtime after this import before running the augmentations
!pip install -U augly
!sudo apt-get install python3-magic

     |████████████████████████████████| 40.1MB 75kB/s 
     |████████████████████████████████| 430kB 24.4MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 27.4MB 116kB/s 
     |████████████████████████████████| 3.0MB 36.9MB/s 
     |████████████████████████████████| 3.2MB 32.3MB/s 
     |████████████████████████████████| 184kB 49.3MB/s 
     |████████████████████████████████| 1.9MB 32.9MB/s 
     |████████████████████████████████| 51.0MB 55kB/s 
     |████████████████████████████████| 399kB 29.6MB/s 
     |████████████████████████████████| 81kB 9.8MB/s 
     |████████████████████████████████| 22.3MB 1.5MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
     |████████████████████████████████| 11.6MB 16.6MB/s 
     |████████████████████████████████| 3.4MB 53.3MB/s 
     |████████████████████████████████| 829kB 26.5MB/s 
     |████████████████████████████████| 645kB 27.8MB/s 
     |████████████████████████████████| 112kB 45.0MB/s 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libmagic-mgc libmagic1
Suggested packages:
  file
The following NEW packages will be installed:
  libmagic-mgc libmagic1 python3-magic
0 upgraded, 3 newly installed, 0 to remove and 39 not upgraded.
Need to get 262 kB of archives.
After this operation, 5,257 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.4 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5.32-2ubuntu0.4 [68.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/main amd64 python3-magic all 2:0.4.15-1 [9,280 B]
Fetched 262 kB in 2s (158 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76

In [2]:
import os
import augly.audio as audaugs
import augly.utils as utils
from IPython.display import display, Audio

# Get input audio
input_audio = os.path.join(
    utils.TEST_URI, "audio", "inputs", "vad-go-stereo-44100.wav"
)
display(Audio(input_audio))

In [3]:
# Now we can apply various augmentations to the audio!
aug_audio, sr = audaugs.pitch_shift(input_audio, n_steps=4.0)
display(Audio(aug_audio, rate=sr))

In [4]:
"""
You can optionally pass in a metadata list, to which metadata about the
augmentation will be appended, including kwargs, the durations & number of
channels of the input & output audio clips, the matching segments in the
input/output audio clips (useful in case of temporal editing), and intensity
(defined based on the kwargs for each augmentation).
"""
meta = []
aug_audio, sr = audaugs.time_stretch(
    input_audio,
    rate=0.5,
    metadata=meta,
)
display(Audio(aug_audio, rate=sr))
meta

[{'dst_duration': 1.965487528344671,
  'dst_num_channels': 2,
  'dst_sample_rate': 44100,
  'dst_segments': [OrderedDict([('start', 0.0), ('end', 1.965487528344671)])],
  'intensity': 20.0,
  'name': 'time_stretch',
  'output_path': None,
  'rate': 0.5,
  'src_duration': 0.9827437641723356,
  'src_num_channels': 2,
  'src_sample_rate': 44100,
  'src_segments': [OrderedDict([('start', 0.0),
                ('end', 0.9827437641723356)])]}]

In [5]:
# For all the augmentations, we have class-based definitions as well as
# functional
from augly.audio.utils import validate_and_load_audio

input_audio_arr, sr = validate_and_load_audio(input_audio)
meta = []
aug = audaugs.PeakingEqualizer()
aug_audio, sr = aug(input_audio_arr, sample_rate=sr, metadata=meta)
display(Audio(aug_audio, rate=sr))
meta

[{'center_hz': 500.0,
  'dst_duration': 0.9827437641723356,
  'dst_num_channels': 2,
  'dst_sample_rate': 44100,
  'dst_segments': [OrderedDict([('start', 0.0), ('end', 0.9827437641723356)])],
  'gain_db': -3.0,
  'intensity': 2.66798418972332,
  'name': 'peaking_equalizer',
  'output_path': None,
  'q': 1.0,
  'src_duration': 0.9827437641723356,
  'src_num_channels': 2,
  'src_sample_rate': 44100,
  'src_segments': [OrderedDict([('start', 0.0),
                ('end', 0.9827437641723356)])]}]

In [6]:
# You can also compose several transformations together
meta = []
aug = audaugs.Compose(
    [
        audaugs.AddBackgroundNoise(),
        audaugs.ToMono(),
        audaugs.Clicks(),
    ]
)
aug_audio, sr = aug(input_audio_arr, sample_rate=sr, metadata=meta)
display(Audio(aug_audio, rate=sr))
meta

[{'background_audio': None,
  'background_duration': 0.9827437641723356,
  'dst_duration': 0.9827437641723356,
  'dst_num_channels': 2,
  'dst_sample_rate': 44100,
  'dst_segments': [OrderedDict([('start', 0.0), ('end', 0.9827437641723356)])],
  'intensity': 90.9090909090909,
  'name': 'add_background_noise',
  'output_path': None,
  'snr_level_db': 10.0,
  'src_duration': 0.9827437641723356,
  'src_num_channels': 2,
  'src_sample_rate': 44100,
  'src_segments': [OrderedDict([('start', 0.0),
                ('end', 0.9827437641723356)])]},
 {'dst_duration': 0.9827437641723356,
  'dst_num_channels': 1,
  'dst_sample_rate': 44100,
  'dst_segments': [OrderedDict([('start', 0.0), ('end', 0.9827437641723356)])],
  'intensity': 100.0,
  'name': 'to_mono',
  'output_path': None,
  'src_duration': 0.9827437641723356,
  'src_num_channels': 2,
  'src_sample_rate': 44100,
  'src_segments': [OrderedDict([('start', 0.0),
                ('end', 0.9827437641723356)])]},
 {'clicks_duration': 0.6,
  '

In [7]:
# Testez d'autres transformations: https://github.com/facebookresearch/AugLy/blob/main/augly/audio/__init__.py

# Others transformations testing on audio files

In [10]:
# You can also compose several transformations together
meta = []
aug = audaugs.Compose(
    [
        audaugs.Speed(),
        audaugs.LowPassFilter(),
        audaugs.Reverb(),
    ]
)
aug_audio, sr = aug(input_audio_arr, sample_rate=sr, metadata=meta)
display(Audio(aug_audio, rate=sr))
meta

[{'dst_duration': 0.4913718820861678,
  'dst_num_channels': 2,
  'dst_sample_rate': 88200,
  'dst_segments': [OrderedDict([('start', 0.0), ('end', 0.4913718820861678)])],
  'factor': 2.0,
  'intensity': 20.0,
  'name': 'speed',
  'output_path': None,
  'src_duration': 0.9827437641723356,
  'src_num_channels': 2,
  'src_sample_rate': 44100,
  'src_segments': [OrderedDict([('start', 0.0),
                ('end', 0.9827437641723356)])]},
 {'alpha': 0.03439388959971595,
  'cutoff_hz': 500.0,
  'dst_duration': 0.4913718820861678,
  'dst_num_channels': 2,
  'dst_sample_rate': 88200,
  'dst_segments': [OrderedDict([('start', 0.0), ('end', 0.4913718820861678)])],
  'intensity': 97.5,
  'name': 'low_pass_filter',
  'output_path': None,
  'src_duration': 0.4913718820861678,
  'src_num_channels': 2,
  'src_sample_rate': 88200,
  'src_segments': [OrderedDict([('start', 0.0),
                ('end', 0.9827437641723356)])]},
 {'dst_duration': 0.4913718820861678,
  'dst_num_channels': 2,
  'dst_sampl